In [ ]:
import numpy as np
import matplotlib
matplotlib.use("pgf")
import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True,  # optionally True if you want LaTeX labels
    "font.family": "Times",
    "font.size": 11,
})

from matplotlib.colors import TwoSlopeNorm
%matplotlib inline
# Grid
x, y = np.meshgrid(np.linspace(-2, 2, 30), np.linspace(-2, 2, 30))

# Define different wind fields
fields = {
    'Zero Vorticity (Uniform Flow)': (np.full_like(x, 2), np.zeros_like(x)),
    'Shear Vorticity': (y, np.zeros_like(x)),
    # 'Curvature Vorticity (Non-divergent)': (
    #     -y / (x**2 + y**2 + 0.1),
    #      x / (x**2 + y**2 + 0.1)
    # ),
    'Nonlinear Shear Vorticity': (y**2, np.zeros_like(x)),
    'Positive Vorticity (Cyclonic)': (-y, x),
    'Negative Vorticity (Anticyclonic)': (y, -x)
}

zeta_all = []

# First pass: calculate all zeta values to determine global limits
for (u, v) in fields.values():
    dx = x[0,1] - x[0,0]
    dy = y[1,0] - y[0,0]
    dvdx = np.gradient(v, axis=1) / dx
    dudy = np.gradient(u, axis=0) / dy
    zeta = dvdx - dudy
    zeta = np.round(zeta, 2)
    zeta_all.append(zeta)

# Global vorticity range
zeta_stack = np.stack(zeta_all)
vmax = np.max(np.abs(zeta_stack))
norm = TwoSlopeNorm(vmin=-vmax, vcenter=0, vmax=vmax)

# Plot
i = 0
for (title, (u, v)), zeta in zip(fields.items(), zeta_all):
    fig, ax = plt.subplots(1, 1, figsize = (3,2) ,constrained_layout=True)
    c = ax.contourf(x, y, zeta, cmap='RdBu', levels=20, norm=norm)
    step = 3  # or 3
    quiver = ax.quiver(x[::step, ::step], y[::step, ::step],
                    u[::step, ::step], v[::step, ::step],
                    scale=30, color='black', alpha=0.8)    # ax.streamplot(x, y, u, v, color='k', density=0.3)
    ax.set_title(title)
    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    if np.all(zeta == zeta[0,0]):
        # c = ax.contourf(x, y, zeta, cmap='RdBu', levels=[zeta[0,0]-1e-6, zeta[0,0]+1e-6], vmin=zeta[0,0], vmax=zeta[0,0])
        cb = fig.colorbar(c, ax=ax, location='right', shrink=1)
        cb.set_ticks([zeta[0,0]])
        cb.set_ticklabels([f'{zeta[0,0]:.2f}'])
        cb.set_label(r'$\zeta$ (s$^{-1}$)')
    else:
        cb = fig.colorbar(c, ax=ax, location='right', label=r'$\zeta$ (s$^{-1}$)', shrink=1)
    # plt.show()
    fig.savefig(f"../images/vorticity_plot{i}.pdf")
    i = i + 1
